In [1]:
import os
import tensorflow as tf
from tensorflow import keras

In [2]:
base_dir = './data/cats_and_dogs'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# train collection
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# validation collection
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
model = keras.models.Sequential([
    # 卷积层1
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)),
    keras.layers.MaxPool2D(2, 2),

    # 卷积层2
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPool2D(2, 2),

    # 卷积层3
    keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPool2D(2, 2),

    # 拉平
    keras.layers.Flatten(),

    # dense层
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(2, activation='softmax'),
])